In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import itertools
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import mmap
from collections import defaultdict
import json
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [2]:
def plot_and_calculate_percentages(histograms):
  # Loop through each histogram in the list
  for i, histogram in enumerate(histograms):
    total_counts = sum(histogram)
    percentages = [count / total_counts * 100 for count in histogram]
    average_percentage = sum(percentages) / len(percentages)

    # Create a bar chart for each histogram
    plt.figure(figsize=(10, 6))  # Adjust figure size for multiple plots
    plt.bar(range(len(percentages)), percentages)
    plt.xlabel("Value")
    plt.ylabel("Percentage (%)")
    plt.title("Histogram {} Percentages (Average = {:.2f}%)".format(i+1, average_percentage))
    plt.grid(True)
    plt.show()



In [3]:
def flatten_and_expand(df, col):
    # Iterate over each row in the specified column
    for index, row in enumerate(df[col]): 
        
        # Flatten basic key-value pairs
        for key, value in row.items():
            if key != 'printabledist':
                df.loc[index, f'{col}_{key}_0'] = value 

        # Expand 'printabledist' into separate columns
        for i, dist_value in enumerate(row['printabledist']):
            df.loc[index, f'{col}_printabledist_{i}_0'] = dist_value

    return df

In [ ]:
df = pd.read_json("train_features_5.jsonl",lines = True)

In [ ]:
df

In [ ]:
df.columns.tolist()

In [ ]:
df[:,'sha256':'md5']

In [ ]:
df.columns

In [ ]:
new_data = {f'byte_{i}': [row[i] for row in df['histogram'].to_list()] for i in range(256)}
new_df = pd.DataFrame(new_data)
df = pd.concat([df, new_df], axis=1)

In [ ]:
new_data = {f'byte_entropy_{i}': [row[i] for row in df['byteentropy'].to_list()] for i in range(256)}
new_df = pd.DataFrame(new_data)
df = pd.concat([df, new_df], axis=1)

In [ ]:
# Step 1
df['total_counts'] = df.loc[:, 'byte_0': 'byte_255'].sum(axis=1)

# Step 2
for col in df.loc[:, 'byte_0': 'byte_255']:
    df[col] = df[col] / df['total_counts']

In [ ]:
# Step 1
df['total_counts'] = df.loc[:, 'byte_entropy_0': 'byte_entropy_255'].sum(axis=1)

# Step 2
for col in df.loc[:, 'byte_entropy_0': 'byte_entropy_255']:
    df[col] = df[col] / df['total_counts']

In [ ]:
df

In [ ]:
df.shape[0]

In [ ]:
df = flatten_and_expand(df, 'strings')

In [ ]:
type(df['strings'][0]['printabledist'])

In [ ]:
df['sha256'][0]

In [ ]:
df['strings'][0]

In [ ]:
df

In [ ]:
df['imports'][0]

In [ ]:
df['section'][2]

In [ ]:
df['label'].value_counts()

In [ ]:
suspicious_df = df[df['label'] == 1]

In [ ]:
# Extract the list of histograms from the DataFrame

histograms = suspicious_df['histogram'].tolist()

print(histograms)
# Plot and calculate percentages for each histogram
#plot_and_calculate_percentages(histograms)

In [ ]:
benign_df = df[df['label'] == 0]

In [ ]:
histograms = benign_df['histogram'].tolist()

# Plot and calculate percentages for each histogram
plot_and_calculate_percentages(histograms)

In [ ]:
df['avclass'].value_counts()